# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on April 13 (with 2-hour grace period) and worth 25 points

### Overview

This milestone builds on your work from Milestone 1. You will:

1. Evaluate baseline models using default settings.
2. Engineer new features and re-evaluate models.
3. Use feature selection techniques to find promising subsets.
4. Select the top 3 models and fine-tune them for optimal performance.

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone. 


In [2]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



## Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling.

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>   df = pd.read_csv("zillow_cleaned.csv")

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:** 

- You will not use the testing set during this milestone — it’s reserved for final evaluation later.
- You will have to redo the scaling step when you introduce new features (which have to be scaled as well).


In [3]:
# Add as many code cells as you need

df = pd.read_csv('zillow_dataset_cleaned.csv')
df = df.drop(columns=["rawcensustractandblock", "parcelid", "propertyzoningdesc", "taxdelinquencyyear"])
df = df.map(lambda x: 1 if x is True else (0 if x is False else x))
tax_flags = []
for i in df["taxdelinquencyflag"]:
    if i == 0:
        tax_flags.append(0)
    else:
        tax_flags.append(1)

df["taxdelinquencyflag"] = tax_flags


df

,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,latitude,...,propertylandusetypeid_261.0,propertylandusetypeid_263.0,propertylandusetypeid_264.0,propertylandusetypeid_265.0,propertylandusetypeid_266.0,propertylandusetypeid_267.0,propertylandusetypeid_269.0,propertylandusetypeid_275.0,regionidcounty_2061.0,regionidcounty_3101.0
0,3.5,4.0,3.5,3100.0,3100.0,0.0,3.0,2.0,633.0,33634931.0,...,1,0,0,0,0,0,0,0,0,0
1,1.0,2.0,1.0,1465.0,1465.0,1.0,1.0,1.0,0.0,34449266.0,...,1,0,0,0,0,0,0,0,1,0
2,2.0,3.0,2.0,1243.0,1243.0,0.0,2.0,2.0,440.0,33886168.0,...,1,0,0,0,0,0,0,0,0,0
3,3.0,4.0,3.0,2376.0,2376.0,0.0,3.0,0.0,0.0,34245180.0,...,1,0,0,0,0,0,0,0,0,1
4,3.0,3.0,3.0,1312.0,1312.0,0.0,3.0,0.0,0.0,34185120.0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70795,3.0,3.0,3.0,1741.0,1741.0,0.0,3.0,0.0,0.0,34202400.0,...,0,0,0,0,1,0,0,0,0,1
70796,2.0,2.0,2.0,1286.0,1286.0,0.0,2.0,0.0,0.0,34245368.0,...,1,0,0,0,0,0,0,0,0,1
70797,2.0,4.0,2.0,1612.0,1612.0,1.0,2.0,2.0,560.0,34300140.0,...,1,0,0,0,0,0,0,0,1,0
70798,1.0,3.0,1.0,1032.0,1032.0,0.0,1.0,0.0,0.0,34040895.0,...,1,0,0,0,0,0,0,0,0,1


In [4]:

df['calculatedbathnbr'] = df['calculatedbathnbr'].fillna(df['calculatedbathnbr'].mean())
df['calculatedfinishedsquarefeet'] = df['calculatedfinishedsquarefeet'].fillna(df['calculatedfinishedsquarefeet'].mean())
df['finishedsquarefeet12'] = df['finishedsquarefeet12'].fillna(df['finishedsquarefeet12'].mean())
df['fullbathcnt'] = df['fullbathcnt'].fillna(df['fullbathcnt'].mean())
df['lotsizesquarefeet'] = df['lotsizesquarefeet'].fillna(df['lotsizesquarefeet'].mean())
df['unitcnt'] = df['unitcnt'].fillna(df['unitcnt'].mean())
df['yearbuilt'] = df['yearbuilt'].fillna(df['yearbuilt'].mean()) 
df = df.dropna(how='any',axis=0) 


In [5]:
print(f"{df.isna().sum()}")

bathroomcnt                     0
bedroomcnt                      0
calculatedbathnbr               0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fireplacecnt                    0
fullbathcnt                     0
garagecarcnt                    0
garagetotalsqft                 0
latitude                        0
longitude                       0
lotsizesquarefeet               0
poolcnt                         0
propertycountylandusecode       0
regionidcity                    0
regionidzip                     0
roomcnt                         0
unitcnt                         0
yearbuilt                       0
taxdelinquencyflag              0
pooltypeid_merged               0
log_taxvaluedollarcnt           0
buildingqualitytypeid_2.0       0
buildingqualitytypeid_3.0       0
buildingqualitytypeid_4.0       0
buildingqualitytypeid_5.0       0
buildingqualitytypeid_6.0       0
buildingqualitytypeid_7.0       0
buildingqualitytypeid_8.0       0
buildingqualit

In [6]:

X = df.drop('log_taxvaluedollarcnt', axis=1)
y = df["log_taxvaluedollarcnt"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

### Part 1: Baseline Modeling [3 pts]

Apply the following regression models to the scaled training dataset using **default parameters**:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each model:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV RMSE Score** across all folds in a table. 


In [8]:
# Add as many code cells as you need
from sklearn.tree import DecisionTreeRegressor

model_scores = {}


linReg = LinearRegression()
ridgeReg = Ridge()
lassoReg = Lasso()
dTreeReg = DecisionTreeRegressor()
bagReg = BaggingRegressor()
rfReg = RandomForestRegressor()
gbReg = GradientBoostingRegressor()

models = [linReg, ridgeReg, lassoReg, dTreeReg, bagReg, rfReg, gbReg]
rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

for model in models:
    model.fit(X_train_scaled, y_train)
    scores = cross_val_score(model, X_train_scaled, y_train, cv=rkf, scoring='neg_mean_squared_error')
    print(f"{scores}")
    model_scores[str(model)] = scores


[-0.4789613  -0.48316995 -0.4862286  -0.49829389 -0.48340499 -0.493215
 -0.47849805 -0.49019376 -0.48842663 -0.48139867 -0.48848897 -0.48825688
 -0.48369082 -0.48804622 -0.48197807 -0.49346006 -0.48774038 -0.47537399
 -0.48631626 -0.48859767 -0.48922546 -0.49036147 -0.48478245 -0.48659174
 -0.4808594 ]
[-0.47895588 -0.4831712  -0.48623318 -0.49825849 -0.48340189 -0.49321546
 -0.47848268 -0.49013827 -0.48842731 -0.48138531 -0.48848747 -0.48824309
 -0.48369173 -0.48803723 -0.48195964 -0.49344239 -0.48774369 -0.4753742
 -0.48631527 -0.48860022 -0.48921876 -0.49035169 -0.48477855 -0.48659784
 -0.48085703]
[-0.77617488 -0.78379693 -0.78234693 -0.78201295 -0.80303225 -0.79563637
 -0.77338198 -0.78462438 -0.79538232 -0.77834019 -0.7943541  -0.78200629
 -0.77972163 -0.78209987 -0.78923187 -0.80014888 -0.76644512 -0.77349222
 -0.79155869 -0.79577531 -0.77171464 -0.80284908 -0.78883213 -0.7836795
 -0.78039813]
[-0.84185398 -0.84015579 -0.84849965 -0.85992772 -0.85065478 -0.81469175
 -0.86618465 

In [9]:
values = {}

for key, value in model_scores.items():
    value = np.sqrt(np.array(-1 * value))
    values[key] = (np.mean(value), np.std(value))

In [10]:
pd.DataFrame(values)

,LinearRegression(),Ridge(),Lasso(),DecisionTreeRegressor(),BaggingRegressor(),RandomForestRegressor(),GradientBoostingRegressor()
0,0.697287,0.697282,0.886257,0.920197,0.694761,0.667247,0.660325
1,0.003629,0.003626,0.005484,0.008384,0.004125,0.003996,0.004104


### Part 1: Discussion [2 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which models perform best overall?
  - Which are most stable (lowest std)?
  - Any signs of overfitting or underfitting?

The model that performed the best was the Gradient Boosting Regressor. The model with the lowest standard deviation was the Ridge Regression model. There are some signs of overfititng with the DecisionTreeRegressor and Lasso Regression with them having the highest RMSE and standard deviations out of all the models. 

### Part 2: Feature Engineering [3 pts]

Consider **at least three new features** based on your Milestone 1, Part 5. Examples include:
- Polynomial terms
- Log or interaction terms
- Groupings or transformations of categorical features

Add these features to `X_train` and then:
- Scale using `StandardScaler` 
- Re-run all models listed above (using default settings again).
- Report updated RMSE scores (mean and std) across repeated CV in a table. 

**Note:**  Recall that this will require creating a new version of the dataset, so effectively you may be running "polynomial regression" using `LinearRegression`. 

In [60]:
df_p2 = df.copy()

In [51]:

# features = df_p2.columns.tolist()

# fig, axes = plt.subplots(nrows=8, ncols=len(features)//8, figsize=(40, 30))

# axes = axes.flatten()

# target = 'log_taxvaluedollarcnt'

# for i, feature in enumerate(features):
#     ax = axes[i]
#     ax.scatter(df_p2[feature], df_p2[target], alpha=0.5)
#     ax.set_title(f'{feature} vs {target}')
#     ax.set_xlabel(feature)
#     ax.set_ylabel(target)

# # Adjust layout
# plt.tight_layout()

In [61]:
def log_transformation(df, columns):
    for col in columns:
        df[f"log_{col}"] = np.log1p(df[col])
    return df
log_col = ['calculatedfinishedsquarefeet', 'finishedsquarefeet12','garagetotalsqft', 'lotsizesquarefeet']
df_p2 = log_transformation(df_p2.copy(), log_col)

In [62]:
df_p2['room_size'] = df_p2['finishedsquarefeet12'] / df_p2['roomcnt']  # Finished sqft per room
# df_p2['garage_size'] = df_p2['garagetotalsqft'] / df_p2['garagecarcnt']  # Finished sqft per garage
df_p2['pool_to_lot_ratio'] = df_p2['poolcnt'] / df_p2['lotsizesquarefeet']  # Pools to lot size ratio


# Getting rid of infinte values
df_p2['room_size'] = df_p2['room_size'].replace(np.inf, 0)
# df_p2['garage_size'] = df_p2['garage_size'].replace(np.inf, 0)


In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Example latitude and longitude data
coords = np.array([[df_p2["latitude"].iloc[i], df_p2["longitude"].iloc[i]] for i in range(0, len(df_p2["latitude"]))])

coords
# # Perform K-means clustering
kmeans = KMeans(n_clusters=6)
kmeans.fit(coords)

# Assign each point to a cluster 
labels = kmeans.labels_
df_p2["coords"] = labels
print("Neighborhood labels:", labels)

Neighborhood labels: [1 2 3 ... 4 3 3]


In [64]:
X = df_p2.drop('log_taxvaluedollarcnt', axis=1)
y = df_p2["log_taxvaluedollarcnt"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [65]:
# Add as many code cells as you need
from sklearn.tree import DecisionTreeRegressor

model_scores_2 = {}


linReg = LinearRegression()
ridgeReg = Ridge()
lassoReg = Lasso()
dTreeReg = DecisionTreeRegressor()
bagReg = BaggingRegressor()
rfReg = RandomForestRegressor()
gbReg = GradientBoostingRegressor()

models = [linReg, ridgeReg, lassoReg, dTreeReg, bagReg, rfReg, gbReg]
rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

for model in models:
    model.fit(X_train_scaled, y_train)
    scores = cross_val_score(model, X_train_scaled, y_train, cv=rkf, scoring='neg_mean_squared_error')
    print(f"{scores}")
    model_scores_2[str(model)] = scores



[-0.47104459 -0.47750439 -0.47972503 -0.49068976 -0.47699196 -0.48520295
 -0.47419837 -0.48371922 -0.47970686 -0.47362736 -0.48086137 -0.48220731
 -0.47824631 -0.48134847 -0.47335177 -0.4864534  -0.47941687 -0.4686793
 -0.47848776 -0.48358957 -0.4787379  -0.4848287  -0.48031598 -0.47945572
 -0.47551327]
[-0.47105628 -0.47750342 -0.47972129 -0.49065727 -0.47698706 -0.48520098
 -0.47418977 -0.48367618 -0.47972549 -0.47361425 -0.48085446 -0.48221163
 -0.47826208 -0.48134349 -0.47333523 -0.48645057 -0.47942658 -0.46867943
 -0.47848731 -0.4835965  -0.47869822 -0.48481656 -0.48031339 -0.47944843
 -0.47551025]
[-0.77617488 -0.78379693 -0.78234693 -0.78201295 -0.80303225 -0.79563637
 -0.77338198 -0.78462438 -0.79538232 -0.77834019 -0.7943541  -0.78200629
 -0.77972163 -0.78209987 -0.78923187 -0.80014888 -0.76644512 -0.77349222
 -0.79155869 -0.79577531 -0.77171464 -0.80284908 -0.78883213 -0.7836795
 -0.78039813]
[-0.8271149  -0.84849247 -0.8464413  -0.84512792 -0.83486352 -0.83463278
 -0.8686926

In [66]:
values = {}

for key, value in model_scores_2.items():
    value = np.sqrt(np.array(-1 * value))
    values[key] = (np.mean(value), np.std(value))

In [67]:
pd.DataFrame(values)

,LinearRegression(),Ridge(),Lasso(),DecisionTreeRegressor(),BaggingRegressor(),RandomForestRegressor(),GradientBoostingRegressor()
0,0.692346,0.692343,0.886257,0.919217,0.693924,0.667404,0.661109
1,0.003539,0.003536,0.005484,0.006658,0.004672,0.003863,0.003862


### Part 2: Discussion [2 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

- Which new features seemed to help — and in which models?

- Do you have any hypotheses about why a particular feature helped (or didn’t)?

- Were there any unexpected results?



None of the models showed significant improvement but the Lienar Regression and Ridge Regression models showed some improvement. Most of the improvements helped the models that were not ensemble models. I think the log transformation features were helpful because they were better able to correlate with the target variable. I believe the room size variable is also helpful since it is able to provide more information about features of the houses.

### Part 3: Feature Selection [3 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features.
- Report updated RMSE scores (mean and std) across repeated CV in a table.


### Part 3: Discussion [2 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?

- Which features were consistently retained across models?

- Were any of your newly engineered features selected as important?

- How did feature selection differ between linear and tree-based models?

> Your text here

### Part 4: Fine-Tuning Your Top 3 Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far.

1. Choose the top 3 models based on performance and interpretability from earlier parts.
2. For each model:
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, or other techniques from previous homeworks. 
   - Experiment with different versions of your feature engineering and preprocessing — treat these as additional tunable components.
3. Report the mean and standard deviation of CV RMSE score for each model in a summary table.



In [5]:
# Add as many code cells as you need

### Part 4: Discussion [4 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?
- Provide a ranking of your three models and explain your reasoning — not just based on RMSE, but also interpretability, training time, or generalizability.
- Conclude by considering whether this workflow has produced the results you expected. Typically, you would repeat steps 2 - 4 and also reconsider the choices you made in Milestone 1 when cleaning the dataset, until reaching the point of diminishing returns; do you think that would that have helped here?

> Your text here